In [ ]:
pip install pymongo

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
pip install pymongo[srv]

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
pip install PyMySQL

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
pip install Flask-SQLAlchemy

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
# Import Statements
import os
import json
import numpy
import datetime
import pandas as pd
import pymongo
from sqlalchemy import create_engine
import pyspark.pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

In [ ]:
jdbc_hostname = "gmf7dpk-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "sakila_dw2"

connection_properties = {
  "user" : "gmf7dpk",
  "password" : "Passw0rd321",
  "driver" : "org.mariadb.jdbc.Driver"
}

atlas_cluster_name = "sandbox.zibbf"
atlas_database_name = "Sakila"
atlas_user_name = "gmf7dpk"
atlas_password = "TheUserForGrant"

dst_database = "sakila_dlh"
base_dir = "dbfs:/FileStore/ds2002-lab06"
data_dir = f"{base_dir}/source_data"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"
database_dir = f"{base_dir}/{dst_database}"
new_data_bronze = f"{database_dir}/fact_new_data/bronze"
new_data_silver = f"{database_dir}/fact_new_data/silver"
new_data_gold = f"{database_dir}/fact_new_data/gold"
new_data_stream_dir = f"{stream_dir}/new_data"


In [ ]:
# Set Up

# ######################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
# ######################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

# ######################################################################################################################
# Use this Function to Create New Collections by Uploading JSON file(s) to the MongoDB Atlas server.
# ######################################################################################################################
def set_mongo_collection(user_id, pwd, cluster_name, db_name, src_file_path, json_files):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.qsabeum.mongodb.net/?retryWrites=true&w=majority"
    client = pymongo.MongoClient(mongo_uri)
    db = client[db_name]
    
    '''Read in a JSON file, and Use It to Create a New Collection'''
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(src_file_path, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)

    client.close()
    
    return result

In [ ]:
%sql
DROP DATABASE IF EXISTS sakila_dlh CASCADE;

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS sakila_dlh
COMMENT "Project 2 Capstone"
LOCATION "dbfs:/FileStore/project-2-capstone/sakila_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "Project 2 Capstone");

In [ ]:
# Retrieving Data from Azure Database for MySQL

%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://gmf7dpk-mysql.mysql.database.azure.com:3306/sakila_dw2",
  dbtable "dim_date",
  user "gmf7dpk",
  password "Passw0rd321"
)


In [ ]:
%sql
USE DATABASE sakila_dlh;

CREATE OR REPLACE TABLE sakila_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/project-2-capstone/sakila_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [ ]:
%sql
SELECT * FROM sakila_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20050101,2005-01-01,2005/01/01,01/01/2005,01/01/2005,7,Saturday,1,1,Weekend,53,January,1,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3
20050102,2005-01-02,2005/01/02,01/02/2005,02/01/2005,1,Sunday,2,2,Weekend,53,January,1,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3
20050103,2005-01-03,2005/01/03,01/03/2005,03/01/2005,2,Monday,3,3,Weekday,1,January,1,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3
20050104,2005-01-04,2005/01/04,01/04/2005,04/01/2005,3,Tuesday,4,4,Weekday,1,January,1,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3
20050105,2005-01-05,2005/01/05,01/05/2005,05/01/2005,4,Wednesday,5,5,Weekday,1,January,1,N,1,2005,2005-01,2005Q1,7,3,2005,2005-07,2005Q3


In [ ]:
# Retrieving Database from MongoDB Atlas Database

%scala
import com.mongodb.spark._

val df_films = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("database", "Sakila").option("collection", "New_Films").load()
.select("Title", "Year", "Rated", "Released", "Runtime", "Genre", "Director", "Writer", "Actors", "Plot", "Language", "Country", "Awards", "Poster", "Metascore", "imdbRating", "imdbVotes", "imdbID", "Type", "Response")

display(df_films)

Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Metascore,imdbRating,imdbVotes,imdbID,Type,Response
Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang",A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.,"English, Spanish","USA, UK",Won 3 Oscars. Another 80 wins & 121 nominations.,http://ia.media-imdb.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg,83,7.9,"890,617",tt0499549,movie,True
I Am Legend,2007,PG-13,14 Dec 2007,101 min,"Drama, Horror, Sci-Fi",Francis Lawrence,"Mark Protosevich (screenplay), Akiva Goldsman (screenplay), Richard Matheson (novel), John William Corrington, Joyce Hooper Corrington","Will Smith, Alice Braga, Charlie Tahan, Salli Richardson-Whitfield","Years after a plague kills most of humanity and transforms the rest into monsters, the sole survivor in New York City struggles valiantly to find a cure.",English,USA,9 wins & 21 nominations.,http://ia.media-imdb.com/images/M/MV5BMTU4NzMyNDk1OV5BMl5BanBnXkFtZTcwOTEwMzU1MQ@@._V1_SX300.jpg,65,7.2,"533,874",tt0480249,movie,True
The Avengers,2012,PG-13,04 May 2012,143 min,"Action, Sci-Fi, Thriller",Joss Whedon,"Joss Whedon (screenplay), Zak Penn (story), Joss Whedon (story)","Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth",Earth's mightiest heroes must come together and learn to fight as a team if they are to stop the mischievous Loki and his alien army from enslaving humanity.,"English, Russian",USA,Nominated for 1 Oscar. Another 34 wins & 75 nominations.,http://ia.media-imdb.com/images/M/MV5BMTk2NTI1MTU4N15BMl5BanBnXkFtZTcwODg0OTY0Nw@@._V1_SX300.jpg,69,8.1,"1,003,301",tt0848228,movie,True
The Wolf of Wall Street,2013,R,25 Dec 2013,180 min,"Biography, Comedy, Crime",Martin Scorsese,"Terence Winter (screenplay), Jordan Belfort (book)","Leonardo DiCaprio, Jonah Hill, Margot Robbie, Matthew McConaughey","Based on the true story of Jordan Belfort, from his rise to a wealthy stock-broker living the high life to his fall involving crime, corruption and the federal government.","English, French",USA,Nominated for 5 Oscars. Another 35 wins & 154 nominations.,http://ia.media-imdb.com/images/M/MV5BMjIxMjgxNTk0MF5BMl5BanBnXkFtZTgwNjIyOTg2MDE@._V1_SX300.jpg,75,8.2,"786,985",tt0993846,movie,True
Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Jonathan Nolan, Christopher Nolan","Ellen Burstyn, Matthew McConaughey, Mackenzie Foy, John Lithgow",A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.,English,"USA, UK",Won 1 Oscar. Another 39 wins & 134 nominations.,http://ia.media-imdb.com/images/M/MV5BMjIxNTU4MzY4MF5BMl5BanBnXkFtZTgwMzM4ODI3MjE@._V1_SX300.jpg,74,8.6,"937,412",tt0816692,movie,True
Game of Thrones,2011–,TV-MA,17 Apr 2011,56 min,"Adventure, Drama, Fantasy",N/A,"David Benioff, D.B. Weiss","Peter Dinklage, Lena Headey, Emilia Clarke, Kit Harington","While a civil war brews between several noble families in Westeros, the children of the former rulers of the land attempt to rise up to power. Meanwhile a forgotten race, bent on destruction, plans to return after thousands of years in the North.",English,"USA, UK",Won 1 Golden Globe. Another 185 wins & 334 nominations.,http://ia.media-imdb.com/images/M/MV5BMjM5OTQ1MTY5Nl5BMl5BanBnXkFtZTgwMjM3NzMxODE@._V1_SX300.jpg,N/A,9.5,"1,046,830",tt0944947,series,True
Vikings,2013–,TV-14,03 Mar 2013,44 min,"Action, Drama, History",N/A,Michael Hirst,"Travis Fimmel, Clive Standen, Gustaf Skarsgård, Katheryn Winnick","The world of the Vikings is brought to life through the journey of Ragnar Lothbrok, the first Viking to emerge from Norse legend and onto the pages of history - a man on the edge of myth.","English, Old English, Norse, Old, Latin","Irel

In [ ]:
%scala
df_films.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_films")

In [ ]:
%sql
SELECT * FROM sakila_dlh.dim_films LIMIT 5

Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Metascore,imdbRating,imdbVotes,imdbID,Type,Response
Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang",A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.,"English, Spanish","USA, UK",Won 3 Oscars. Another 80 wins & 121 nominations.,http://ia.media-imdb.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg,83,7.9,"890,617",tt0499549,movie,True
I Am Legend,2007,PG-13,14 Dec 2007,101 min,"Drama, Horror, Sci-Fi",Francis Lawrence,"Mark Protosevich (screenplay), Akiva Goldsman (screenplay), Richard Matheson (novel), John William Corrington, Joyce Hooper Corrington","Will Smith, Alice Braga, Charlie Tahan, Salli Richardson-Whitfield","Years after a plague kills most of humanity and transforms the rest into monsters, the sole survivor in New York City struggles valiantly to find a cure.",English,USA,9 wins & 21 nominations.,http://ia.media-imdb.com/images/M/MV5BMTU4NzMyNDk1OV5BMl5BanBnXkFtZTcwOTEwMzU1MQ@@._V1_SX300.jpg,65,7.2,"533,874",tt0480249,movie,True
The Avengers,2012,PG-13,04 May 2012,143 min,"Action, Sci-Fi, Thriller",Joss Whedon,"Joss Whedon (screenplay), Zak Penn (story), Joss Whedon (story)","Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth",Earth's mightiest heroes must come together and learn to fight as a team if they are to stop the mischievous Loki and his alien army from enslaving humanity.,"English, Russian",USA,Nominated for 1 Oscar. Another 34 wins & 75 nominations.,http://ia.media-imdb.com/images/M/MV5BMTk2NTI1MTU4N15BMl5BanBnXkFtZTcwODg0OTY0Nw@@._V1_SX300.jpg,69,8.1,"1,003,301",tt0848228,movie,True
The Wolf of Wall Street,2013,R,25 Dec 2013,180 min,"Biography, Comedy, Crime",Martin Scorsese,"Terence Winter (screenplay), Jordan Belfort (book)","Leonardo DiCaprio, Jonah Hill, Margot Robbie, Matthew McConaughey","Based on the true story of Jordan Belfort, from his rise to a wealthy stock-broker living the high life to his fall involving crime, corruption and the federal government.","English, French",USA,Nominated for 5 Oscars. Another 35 wins & 154 nominations.,http://ia.media-imdb.com/images/M/MV5BMjIxMjgxNTk0MF5BMl5BanBnXkFtZTgwNjIyOTg2MDE@._V1_SX300.jpg,75,8.2,"786,985",tt0993846,movie,True
Interstellar,2014,PG-13,07 Nov 2014,169 min,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Jonathan Nolan, Christopher Nolan","Ellen Burstyn, Matthew McConaughey, Mackenzie Foy, John Lithgow",A team of explorers travel through a wormhole in space in an attempt to ensure humanity's survival.,English,"USA, UK",Won 1 Oscar. Another 39 wins & 134 nominations.,http://ia.media-imdb.com/images/M/MV5BMjIxNTU4MzY4MF5BMl5BanBnXkFtZTgwMzM4ODI3MjE@._V1_SX300.jpg,74,8.6,"937,412",tt0816692,movie,True


In [ ]:
# Retrieving Data from Azure Databricks File System (DBFS)

candidates_csv = f"/FileStore/ds2002-lab06/sakila_dlh/50-contacts.csv"

df_candidates = spark.read.format('csv').options(header='true', inferSchema='true').load(candidates_csv)
display(df_candidates)

first_name,last_name,company_name,address,city,county,state,zip,phone1,phone,email
James,Butt,"Benton, John B Jr",6649 N Blue Gum St,New Orleans,Orleans,LA,70116,504-621-8927,504-845-1427,jbutt@gmail.com
Josephine,Darakjy,"Chanay, Jeffrey A Esq",4 B Blue Ridge Blvd,Brighton,Livingston,MI,48116,810-292-9388,810-374-9840,josephine_darakjy@darakjy.org
Art,Venere,"Chemel, James L Cpa",8 W Cerritos Ave #54,Bridgeport,Gloucester,NJ,8014,856-636-8749,856-264-4130,art@venere.org
Lenna,Paprocki,Feltz Printing Service,639 Main St,Anchorage,Anchorage,AK,99501,907-385-4412,907-921-2010,lpaprocki@hotmail.com
Donette,Foller,Printing Dimensions,34 Center St,Hamilton,Butler,OH,45011,513-570-1893,513-549-4561,donette.foller@cox.net
Simona,Morasca,"Chapman, Ross E Esq",3 Mcauley Dr,Ashland,Ashland,OH,44805,419-503-2484,419-800-6759,simona@morasca.com
Mitsue,Tollner,Morlong Associates,7 Eads St,Chicago,Cook,IL,60632,773-573-6914,773-924-8565,mitsue_tollner@yahoo.com
Leota,Dilliard,Commercial Press,7 W Jackson Blvd,San Jose,Santa Clara,CA,95111,408-752-3500,408-813-1105,leota@hotmail.com
Sage,Wieser,Truhlar And Truhlar Attys,5 Boston Ave #88,Sioux Falls,Minnehaha,SD,57105,605-414-2147,605-794-4895,sage_wieser@cox.net
Kris,Marrier,"King, Christopher A Esq",228 Runamuck Pl #2808,Baltimore,Baltimore City,MD,21224,410-655-8723,410-804-4694,kris@gmail.com


In [ ]:
df_candidates.write.format("delta").mode("overwrite").saveAsTable("sakila_dlh.dim_candidates")

In [ ]:
# Bronze Table
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaHints", "Index STRING")
 .option("cloudFiles.schemaHints", "User_Id STRING")
 .option("cloudFiles.schemaHints", "First_Name STRING")
 .option("cloudFiles.schemaHints", "Last_Name STRING") 
 .option("cloudFiles.schemaHints", "Sex STRING")
 .option("cloudFiles.schemaHints", "Email STRING")
 .option("cloudFiles.schemaHints", "Phone STRING")
 .option("cloudFiles.schemaHints", "Date_of_birth STRING")
 .option("cloudFiles.schemaHints", "Job_Title STRING")
 .option("cloudFiles.schemaLocation", new_data_bronze)
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(new_data_stream_dir)
 .createOrReplaceTempView("new_data_raw_tempview"))

In [ ]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW new_data_bronze_tempview AS (
  SELECT *, current_timestamp() receipt_time, input_file_name() source_file
  FROM new_data_raw_tempview
)

In [ ]:
(spark.table("new_data_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{new_data_bronze}/_checkpoint")
      .outputMode("append")
      .table("fact_new_data_bronze"))

In [ ]:
(spark.readStream
  .table("new_data_bronze")
  .createOrReplaceTempView("new_data_silver_tempview"))

In [ ]:
# Silver Table
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_new_data_silver_tempview AS (
  SELECT c.first_name,
  c.last_name,
  c.company_name,
  nd.Email,
  nd.Phone,
  nd.JobTitle,
FROM new_data_silver_tempview VERSION AS nd
OUTER JOIN sakila_dlh.dim_candidates AS c
ON nd.Last_Name = c.last_name
}

In [ ]:
(spark.table("fact_new_data_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", f"{new_data_silver}/_checkpoint")
      .outputMode("append")
      .table("fact_new_data_silver"))

In [ ]:
# Gold Table

In [ ]:
%sql
SELECT COUNT(*), JobTitle
FROM sakila_dlh.fact_new_data_silver
GROUP BY JobTitle